In [9]:
from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl.styles import NamedStyle, Font, PatternFill, Alignment
import os

# Die folgenden imports stammen aus der Datei ../00_common/common.py

import sys
sys.path.append("../00_common")
import common

def check_fingerprint(wb, matr):
    ws = wb.worksheets[0]
    seed = "abc"
    val = hash(str(matr) + seed) % 10000000000
    if ws["BB3"].value == val:
        return "OK"
    else:
        return "NOK"

def check_arrayformula(wb, matr):
    try:
        response = "          "
        ws = wb.worksheets[0]
        if ws.array_formulae != {}:
            response = response[:1] + "1" + response[2:]
        
        ws = wb.worksheets[1]
        if ws.array_formulae != {}:
            response = response[:3] + "2" + response[4:]

        ws = wb.worksheets[2]
        if ws.array_formulae != {}:
            response = response[:5] + "3" + response[6:]

        return response
    except:
        print("FAIL:", matr, "check Matrix_Formula")
        return "NOK"

def check_exercise_1_1(wb, wb_val, matr):
    points = 0
    try:
        ws = wb.worksheets[0]
        ws_val = wb_val.worksheets[0]

        if not(isinstance(ws["B2"].value, int)):
            year = ws["B2"].value.year
            month = ws["B2"].value.month
            day = ws["B2"].value.day
        else:
            year = 0
            month = 0
            day = 0

        if "=YEAR" in ws["B4"].value and "=MONTH"  in ws["B5"].value and "=DAY" in ws["B6"].value:
            points += 1
        if year == ws_val["B4"].value and month == ws_val["B5"].value and day == ws_val["B6"].value:
            points += 1
        if "WEEKDAY" in ws["B7"].value and "VLOOKUP" in ws["B7"].value:
            points += 1
        if "WEEKNUM" in ws["B8"].value and "ROUND" in ws["B9"].value:
            points += 1
        if "Version" in ws["B11"].number_format and "dddd" in ws["B11"].number_format and "mmmm" in ws["B11"].number_format:
            points += 1

        if "=B2" in ws["C4"].value and "=B2" in ws["C5"].value and "=B2" in ws["C6"].value and "=B2" in ws["C7"].value:
            points += 1
        if "=B$2" in ws["C4"].value and "=B$2" in ws["C5"].value and "=B$2" in ws["C6"].value and "=B$2" in ws["C7"].value:
            points += 1
        if "=$B$2" in ws["C4"].value and "=$B$2" in ws["C5"].value and "=$B$2" in ws["C6"].value and "=$B$2" in ws["C7"].value:
            points += 1
        if ws_val["C4"].number_format == "yyyy" and (ws_val["C5"].number_format == "m" or ws_val["C5"].number_format == "mm") and \
           (ws_val["C6"].number_format == "d" or ws_val["C6"].number_format == "dd") and ws_val["C7"].number_format == "dddd":
            points += 1

        if not(isinstance(ws["F2"].value, float)):
            hour = ws["F2"].value.hour
            minute = ws["F2"].value.minute
            second = ws["F2"].value.second
        else:
            hour = 0
            minute = 0
            second = 0

        if "=HOUR" in ws["F4"].value and "=MINUTE"  in ws["F5"].value and "=SECOND" in ws["F6"].value:
            points += 1
        if hour == ws_val["F4"].value and minute == ws_val["F5"].value and second == ws_val["F6"].value:
            points += 1
        if (ws_val["G4"].number_format == "h" or ws_val["G4"].number_format == "hh") and \
           (ws_val["G6"].number_format == "s" or ws_val["G6"].number_format == "ss"):
            points += 1
        
        return points
    except:
        print("FAIL:", matr, "check_exercise_1_1")
        return "NOK"

def check_exercise_1_2(wb, wb_val, matr):
    points = 0
    try:
        ws = wb.worksheets[1]
        ws_val = wb_val.worksheets[1]

        if "D9" in ws.array_formulae:
            formula = ws["D9"].value.text
        elif ws["D9"].value != None:
            formula = ws["D9"].value
        else:
            formula = ""
        if "DATEDIF" in formula:
            points += 3

        if "H9" in ws.array_formulae:
            formula = ws["H9"].value.text
        elif ws["H9"].value != None:
            formula = ws["H9"].value
        else:
            formula = ""
        if "IF" in formula:
            points += 2
        if ("B4" in formula or "B$4" in formula) and ("B5" in formula or "B$5" in formula) and ("B6" in formula or "B$6" in formula):
            points += 1

        p1 = ws_val["B4"].value
        p2 = ws_val["B5"].value
        p3 = ws_val["B6"].value
        p_total = 0
        row_num = 9
        while True:
            age = ws_val.cell(row = row_num, column = 4).value
            if not isinstance(age, int):
                break
            if age < 16:
                p_total += p1
            elif age < 18:
                p_total += p2
            else:
                p_total += p3
            row_num += 1
        if ws_val["E1"].value == p_total:
            points += 4

        return points
    except:
        print("FAIL:", matr, "check_exercise_1_2")
        return "NOK"


def check_exercise_1_3(wb, matr):
    points = 0
    try:
        ws = wb.worksheets[2]
        # Check format
        if ".00" in ws["C5"].number_format and "€" in ws["C5"].number_format and "Red" in ws["C5"].number_format:
            points += 1
        if ".0%" in ws["F4"].number_format:
            points += 1
        
        return points
    except:
        print("FAIL:", matr, "check_exercise_1_3")
        return "NOK"

def check_exercise_1_4(wb, matr):
    points = 0
    try:
        ws = wb.worksheets[2]

        if "C5" in ws.array_formulae:
            formula = ws["C5"].value.text
        elif ws["C5"].value != None:
            formula = ws["C5"].value
        else:
            formula = ""
        if "DAYS360" in formula:
            points += 2

        if "C19" in ws.array_formulae:
            formula = ws["C19"].value.text
        elif ws["C19"].value != None:
            formula = ws["C19"].value
        else:
            formula = ""
        if "366" in formula:
            points += 1
        if "$B$2" in formula:
            points += 1    

        return points
    except:
        print("FAIL:", matr, "check_exercise_1_4")
        return "NOK"

def check_exercise_1_5(wb_val, matr):
    points = 0
    try:
        ws_val = wb_val.worksheets[2]

        if ws_val["C5"].value != None and ws_val["C7"].value != None and ws_val["C10"].value != None:
            if round(ws_val["C5"].value * 6, 2) == round(ws_val["C7"].value * 2, 2) == round(ws_val["C10"].value, 2):
                points += 2

        if ws_val["D16"].value != None and ws_val["D30"].value != None:
            if round(ws_val["D16"].value, 2) == round(ws_val["D30"].value, 2):
                points += 2

        if ws_val["I16"].value != None:
            if round(ws_val["B1"].value * ws_val["I4"].value, 2) == round(ws_val["I16"].value, 2):
                points += 2

        return points
    except:
        print("FAIL:", matr, "check_exercise_1_5")
        return "NOK"

def check_workbook(matr, wb, wb_val):
    points = [matr]
    points.append(check_fingerprint(wb, matr))
    points.append(check_arrayformula(wb, matr))
    points.append(check_exercise_1_1(wb, wb_val, matr))
    points.append(check_exercise_1_2(wb, wb_val, matr))
    points.append(check_exercise_1_3(wb, matr))
    points.append(check_exercise_1_4(wb, matr))
    points.append(check_exercise_1_5(wb_val, matr))
    #points.append(check_exercise_2_2(wb, matr))
    #points.append(check_exercise_2_3(wb_val, matr))
    #points.append(check_exercise_3_1(wb, matr))
    #points.append(check_exercise_3_2(wb_val, matr))
    #points.append(check_exercise_3_3(wb_val, matr))
    #points.append(check_exercise_4_1(wb, wb_val, matr))
    
    return points



l_points = common.check_ExcelFiles_in_Abgaben(check_workbook)

for points in l_points:
    print(points)
    
    header = ["MatrNr", "Fingerprint", "Matrix-F", "1 Num_Form", "1 Address", "1 Products", "1 Calc", "2 Format", "2 Formula", "2 Value", "3 Formula", "3 Domains", "3 Counts", "4 Logic"] 
common.result_into_file(l_points, header)


[3638125, 'NOK', '          ', 8, 9, 2, 4, 6]
[3639940, 'NOK', '          ', 8, 9, 2, 4, 6]


In [8]:
os.chdir(".")
print(os.getcwd())

/Users/jacobs/GIT/PC_Praktikum/09_Excel


G2 + G3 - G4
